In [2]:
!pip install tensorflow

In [3]:
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Dropout, RepeatVector, Embedding , TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.losses import sparse_categorical_crossentropy


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
!git clone https://github.com/ehsanh123/NLP-Week6.git

fatal: destination path 'NLP-Week6' already exists and is not an empty directory.


In [6]:
# path ='/content/drive/MyDrive/NLP/spa.txt'
path = '/content/NLP-Week6/spa.txt'
traslation_file = open(path ,"r",encoding="utf-8")
raw_data = traslation_file.read()
traslation_file.close()

In [7]:
from typing import Sequence
raw_data = raw_data.split('\n')
pairs = [Sequence.split('\t') for Sequence in raw_data]
pairs = pairs[1000:20000]

In [8]:
pairs[100]

['Tom cares.',
 'Tom se preocupa.',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #2203611 (CK) & #4557602 (swyter)']

In [9]:
def clean_sentence(sentece):
  lower_case = sentece.lower()
  string_punctuation = string.punctuation + "¿"
  clean_sentence = lower_case.translate(str.maketrans("","",string_punctuation))
  return clean_sentence

In [10]:
def tokenizer(sentece):
  text_tokenizer = Tokenizer()
  text_tokenizer.fit_on_texts(sentece)
  return text_tokenizer.texts_to_sequences(sentece), text_tokenizer

In [11]:
# Clean sentences
english_sentences = [clean_sentence(pair[1]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[0]) for pair in pairs]
# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenizer(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenizer(english_sentences)
print('Maximum length spanish sentence: {}'.format (len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format (len (max(eng_text_tokenized, key=len))))
# Check language length
spanish_vocab = len(spa_text_tokenizer. word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of () unique words".format (spanish_vocab))
print ("English vocabulary is of f) unique words".format (english_vocab))

Maximum length spanish sentence: 5
Maximum length english sentence: 9
Spanish vocabulary is of () unique words
English vocabulary is of f) unique words


In [12]:
max_spanish_len = int(len(max(spa_text_tokenized, key=len)))

max_english_len = int(len(max(eng_text_tokenized, key=len)))

spa_pad_sentence = pad_sequences (spa_text_tokenized, max_spanish_len, padding = "post")

eng_pad_sentence = pad_sequences (eng_text_tokenized, max_english_len, padding = "post")

#Reshape data

spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)

eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [13]:
input_sequence = Input(shape=(max_spanish_len,))

embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)

encoder = LSTM (64, return_sequences=False) (embedding)

r_vec = RepeatVector(max_english_len) (encoder)

decoder = LSTM (64, return_sequences=True, dropout=0.2) (r_vec)

logits = TimeDistributed (Dense(english_vocab)) (decoder)

In [14]:
enc_dec_model = Model(input_sequence, Activation('softmax') (logits))

# enc_dec_model.compile(loss=sparse_categorical_crossentropy,
# optimizer=Adam(1e-3), metrics=['accuracy'])

enc_dec_model.compile(loss=sparse_categorical_crossentropy,
optimizer=Adam(1e-3), metrics=['accuracy'])

enc_dec_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 5, 128)              │         475,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 9, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 9, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 9, 7423)             │         482,495 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 9, 7423)             │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,040,831 (3.97 MB)

 Trainable params: 1,040,831 (3.97 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from tensorflow.keras.models import load_model

enc_dec_model = load_model("/content/NLP-Week6/model.keras")  # Loads the entire model

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [16]:
# model_results = enc_dec_model.fit(spa_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)

In [17]:
# Example test Sentences
test_sentences = [
"How are you",
"Today is Friday",
"This is NLP"
]

# Clean and Tokenize Test Sentences
clean_test_sentences = [clean_sentence (sentence) for sentence in test_sentences]

#Tokenize Test Sentences

test_sequences_spa = spa_text_tokenizer.texts_to_sequences(clean_test_sentences)

test_pad_sequences_spa = pad_sequences(test_sequences_spa, max_spanish_len, padding="post")

# Reshape Test Data
test_pad_sequences_spa = test_pad_sequences_spa.reshape(*test_pad_sequences_spa.shape, 1)

print(test_pad_sequences_spa)

#Predict Translations

predicted_sequences = enc_dec_model.predict(test_pad_sequences_spa)

#Convert Indices to Words

def sequences_to_text(sequences, tokenizer):

  word_index = tokenizer.word_index

  reverse_word_index = dict([(index, word) for (word, index) in word_index.items()])

  return ' '.join([reverse_word_index.get(i,'?') for i in sequences])

# Display Results

for i, sentence in enumerate(test_sentences):
  print("Input Sentence: (sentence)")
  print(f"Predicted Translation: (sequences_to_text(np.argmax(predicted_sequences[i], axis-1), eng_text_tokenizer))")
  print()

[[[  61]
  [  14]
  [   3]
  [   0]
  [   0]]

 [[ 154]
  [   4]
  [1322]
  [   0]
  [   0]]

 [[  10]
  [   4]
  [   0]
  [   0]
  [   0]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Input Sentence: (sentence)
Predicted Translation: (sequences_to_text(np.argmax(predicted_sequences[i], axis-1), eng_text_tokenizer))

Input Sentence: (sentence)
Predicted Translation: (sequences_to_text(np.argmax(predicted_sequences[i], axis-1), eng_text_tokenizer))

Input Sentence: (sentence)
Predicted Translation: (sequences_to_text(np.argmax(predicted_sequences[i], axis-1), eng_text_tokenizer))



In [18]:
# enc_dec_model.save("model.keras")  # Saves the entire model in HDF5 format


#part 2 pre train models

In [19]:
from transformers import MarianMTModel, MarianTokenizer

# Define model name

model_name = "Helsinki-NLP/opus-mt-en-fr"

# Load tokenizer and model

tokenizer = MarianTokenizer.from_pretrained(model_name)

model = MarianMTModel.from_pretrained (model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [21]:
import torch

def translate_text(text, model, tokenizer):

  #Tokenize input text

  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True) # Changed 'return_tensora' to 'return_tensors'

  #Generate translation output

  with torch.no_grad():

    translated = model.generate(**inputs)

  #Decode translated text

  return tokenizer.decode(translated[0], skip_special_tokens=True) # Changed 'skip_special_tokena' to 'skip_special_tokens'

# Example Translation

english_text = "Hello, how are you?"

french_translation = translate_text(english_text, model, tokenizer)

print (f"English: {english_text}") # Changed '(english_text)' to '{english_text}' for proper f-string formatting

print (f"French: {french_translation}") # Changed '(french translation)' to '{french_translation}' for proper f-string formatting

English: Hello, how are you?
French: Bonjour, comment allez-vous ?


In [23]:
sentences = [

"I love artificial intelligence.",

"The encoder-decoder model is widely used in NLP.",

"Hugging Face provides state-of-the-art translation models."]

translations = [translate_text(sentence, model, tokenizer) for sentence in sentences]

for en, fr in zip(sentences, translations): print (f"English: {en}\nFrench: {fr}\n")

English: I love artificial intelligence.
French: J'aime l'intelligence artificielle.

English: The encoder-decoder model is widely used in NLP.
French: Le modèle de codeur-décodeur est largement utilisé dans NLP.

English: Hugging Face provides state-of-the-art translation models.
French: Hugging Face fournit des modèles de traduction à la fine pointe de la technologie.



In [34]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load BART, model

model.name = "facebook/mbart-large-50-many-to-many-mmt"

# tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name) # Use correct model_name for tokenizer

model = MBartForConditionalGeneration.from_pretrained(model_name)
# model = MBartForConditionalGeneration.from_pretrained(model_name)

# * Specify source & target languages

tokenizer.src_lang = "en XX" # Source: English

target_lang = "fr_XX" # Target: French

In [38]:
def translate_mbart(text, model, tokenizer, target_lang):

  # Tokenize input text

  inputs = tokenizer (text, return_tensors="pt", padding=True,
  truncation=True)

  # Generate translation output

  with torch.no_grad():
    translated = model.generate(**inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id [target_lang])
  # Decode translation
  return tokenizer.decode (translated[0], skip_special_tokens=True)

In [39]:
# Example Usage

text = "Artificial intelligence is transforming the world."

translated_text = translate_mbart(text, model, tokenizer, target_lang)

print (f"French: {translated_text}")

French: L'intelligence artificielle transforme le monde.
